# Analyse masked tif videos for tracked cell's mean pixel intensity

You can choose a channel and keep track of it while the recording progresses. 

Some plotting allows to identify the cell-tracking index for brightest cells.

Antoine

In [ ]:
import warnings
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from scipy.ndimage import zoom
import os
from matplotlib import cm

# === Config ===
original_path = "series003_cCAR_tumor.tif"
track_dir = "tracked"

# Channels to show (max 3); change these as needed
channels_to_display = [0, 1, 2]
actin_tubulin = 1  # Channel used for brightness_actin_tubulin extraction
csfe_channel = 0  # Channel used for CSFE extraction

# === Load Data ===
original_stack = tifffile.imread(original_path)  # shape: (T, C, H, W)
track_files = sorted([f for f in os.listdir(track_dir) if f.endswith('.tif') and 'man_track' in f])
tracked_masks = np.array([tifffile.imread(os.path.join(track_dir, f)) for f in track_files])  # shape: (T, H, W)

# === Resize images to match masks ===
zoom_factors = (1, 1, 706 / 1412, 706 / 1412)
resized_original = zoom(original_stack, zoom_factors, order=1)

# === Normalize to [0, 1] per channel ===
norm_resized = np.zeros_like(resized_original, dtype=np.float32)
for c in range(resized_original.shape[1]):
    ch = resized_original[:, c]
    norm_resized[:, c] = (ch - ch.min()) / (ch.max() - ch.min() + 1e-8)

# === Data containers for analysis ===
trajectories = {}
brightness_actin_tubulin = {}
brightness_csfe = {}  # Container for CSFE brightness

# === Set up plot ===
plt.rcParams['animation.embed_limit'] = 100  # Try to make bigger animations work

# Initialize the figure and axes
fig, axes = plt.subplots(1, 2, figsize=(14, 7))
img1 = axes[0].imshow(np.zeros((706, 706, 3)))
axes[0].set_title("Original Image")
axes[0].axis('off')

img2 = axes[1].imshow(np.zeros((706, 706, 3)))  # expects an image
axes[1].set_title(f"Tracked Mask + Channel {actin_tubulin}")
axes[1].axis('off')

title = fig.suptitle("Frame 0")
label_texts = []

# === Animation update ===
def update(i):
    global label_texts
    for t in label_texts:
        t.remove()
    label_texts = []

    # Build multichannel frame
    selected_channels = norm_resized[i, channels_to_display]
    if selected_channels.shape[0] == 1:
        multichan_frame = np.repeat(selected_channels, 3, axis=0).transpose(1, 2, 0)
    elif selected_channels.shape[0] == 2:
        ch0, ch1 = selected_channels
        ch2 = np.zeros_like(ch0)
        multichan_frame = np.stack([ch0, ch1, ch2], axis=-1)
    else:
        multichan_frame = selected_channels[:3].transpose(1, 2, 0)

    img1.set_data(multichan_frame)

    # === Colorize mask by intensity ===
    img_actin_tubulin = norm_resized[i, actin_tubulin]
    img_csfe = norm_resized[i, csfe_channel]  # CSFE channel image
    mask = tracked_masks[i]
    ids = np.unique(mask)
    ids = ids[ids != 0]

    color_mask = np.zeros((*mask.shape, 3), dtype=np.float32)

    for label_id in ids:
        yx = np.argwhere(mask == label_id)
        if len(yx) > 0:
            y_mean, x_mean = yx.mean(axis=0)

            # Trajectory collection
            trajectories.setdefault(label_id, []).append((x_mean, y_mean))

            # Brightness collection for actin-tubulin
            pixels_actin_tubulin = img_actin_tubulin[mask == label_id]
            mean_intensity_actin_tubulin = pixels_actin_tubulin.mean()
            brightness_actin_tubulin.setdefault(label_id, []).append(mean_intensity_actin_tubulin)

            # Brightness collection for CSFE
            pixels_csfe = img_csfe[mask == label_id]
            mean_intensity_csfe = pixels_csfe.mean()
            brightness_csfe.setdefault(label_id, []).append(mean_intensity_csfe)

            # Color by brightness_actin_tubulin
            color = cm.viridis(np.clip(mean_intensity_actin_tubulin / 0.05, 0, 1))[:3]  # Adjust as needed
            color_mask[mask == label_id] = color

            # Label overlay with CSFE brightness as additional information
            txt = axes[1].text(x_mean, y_mean, f"{label_id}\nCSFE: {mean_intensity_csfe:.2f}",
                               color='white', fontsize=8, ha='center', va='center',
                               bbox=dict(facecolor='black', alpha=0.5, edgecolor='none', pad=1))
            label_texts.append(txt)

    img2.set_data(color_mask)
    title.set_text(f"Frame {i}")
    return [img1, img2, title] + label_texts

# === Animate ===
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    anim = FuncAnimation(fig, update, frames=len(tracked_masks), interval=200, blit=False, repeat=False)

plt.close(fig)  # just clear last static pic
HTML(anim.to_jshtml())


## Using napari_ctc

We can obtain a precise tracking of the cells using napari

In [2]:
from pathlib import Path
from napari_ctc_io.reader import read_ctc
masks, tracks, tracks_graph = read_ctc(
    Path(r"tracked")
)


INFO:napari_ctc_io.reader:Loaded tracks from tracked/man_track.txt
Loading segmentations:  36%|███▌      | 58/162 [00:00<00:00, 122.55it/s]

Computing centroids: 100%|██████████| 162/162 [00:02<00:00, 61.14it/s]
INFO:napari_ctc_io.reader:Running CTC format checks
Checking parent links: 1242it [00:00, 23659.39it/s]
Checking for non-connected masks: 100%|██████████| 162/162 [00:01<00:00, 161.47it/s]
INFO:napari_ctc_io.reader:Checks completed


## Cell film making

Using the position obtained from the tracking and the average cell brightness we can track those metrics accross the life of the cell.

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

cell_id = 104
box_size = 80
half_box = box_size // 2
actin_tubulin = 1
csfe_channel = 0  # Assuming CSFE is channel 0
channels_to_display = [0, 1, 3]

# Create DataFrame for this specific cell
tracks_df = pd.DataFrame(tracks, columns=["label", "frame", "x", "y"])
tracks_df = tracks_df[tracks_df["label"] == cell_id]

starting_frame = int(tracks_df["frame"].min())
end_frame = int(tracks_df["frame"].max())

# Brightness for this cell
cell_brightness_actin_tubulin = brightness_actin_tubulin[cell_id]
cell_brightness_csfe = brightness_csfe[cell_id]  # Add the CSFE brightness

# Setup subplots with black background
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4), facecolor='black')  # Adding ax3 for Actin-Tubulin plot
ax1.set_facecolor('black')
ax2.set_facecolor('black')
ax3.set_facecolor('black')

# Plot for CSFE channel
line_csfe, = ax2.plot(cell_brightness_csfe, label=f"Cell {cell_id} (CSFE)", color='magenta')
dot_csfe, = ax2.plot([], [], 'go')
ax2.set_title("Brightness Over Time for CSFE Channel", color='white')
ax2.set_xlabel("Frame", color='white')
ax2.set_ylabel(f"Mean Intensity (Channel {csfe_channel})", color='white')
ax2.grid(True, color='gray')
ax2.tick_params(axis='both', colors='white')

# Plot for actin_tubulin channel (now in ax3)
line_actin_tubulin, = ax3.plot(cell_brightness_actin_tubulin, label=f"Cell {cell_id} (Actin-Tubulin)", color='cyan')
dot_actin_tubulin, = ax3.plot([], [], 'ro')
ax3.set_title("Brightness Over Time for caspase", color='white')
ax3.set_xlabel("Frame", color='white')
ax3.set_ylabel(f"Mean Intensity (Channel {actin_tubulin})", color='white')
ax3.grid(True, color='gray')
ax3.tick_params(axis='both', colors='white')

def update(frame):
    frame = int(frame) + starting_frame
    ax1.clear()
    ax1.set_facecolor('black')

    ax2.set_xlim(starting_frame, end_frame)
    ax2.set_ylim(0, max(cell_brightness_csfe) * 1.1)

    ax3.set_xlim(starting_frame, end_frame)
    ax3.set_ylim(0, max(cell_brightness_actin_tubulin) * 1.1)

    # === Image Crop ===
    cell_position = tracks_df[tracks_df["frame"] == frame]
    if not cell_position.empty:
        x, y = cell_position.iloc[0][["x", "y"]]
        x, y = int(x), int(y)

        x_start = max(x - half_box, 0)
        x_end = min(x + half_box, resized_original.shape[2])
        y_start = max(y - half_box, 0)
        y_end = min(y + half_box, resized_original.shape[3])

        crop = resized_original[frame, :, x_start:x_end, y_start:y_end]
        crop = crop[channels_to_display]

        ax1.imshow(np.transpose(crop, (1, 2, 0)))
        ax1.scatter(half_box, half_box, color='red', s=50)
        ax1.set_title(f"Cell {cell_id} - Frame {frame}", color='white')
        ax1.axis('off')

    # === Brightness Highlight for CSFE ===
    line_csfe.set_data(range(starting_frame, end_frame + 2), cell_brightness_csfe)
    dot_csfe.set_data([frame], [cell_brightness_csfe[frame - starting_frame]])

    # === Brightness Highlight for Actin-Tubulin ===
    line_actin_tubulin.set_data(range(starting_frame, end_frame + 2), cell_brightness_actin_tubulin)
    dot_actin_tubulin.set_data([frame], [cell_brightness_actin_tubulin[frame - starting_frame]])

    return [line_csfe, dot_csfe, line_actin_tubulin, dot_actin_tubulin]

ani = FuncAnimation(fig, update, frames=end_frame - starting_frame, interval=200, blit=False)

plt.close(fig)  # just clear last static pic

# For notebook display
from IPython.display import HTML
HTML(ani.to_jshtml())


We can then correllate the lowering of the CSFE brightness and the increasing in actin-tubuling channel with cancer cell death.

In [5]:
ani.save("cell_animation.gif", writer='ffmpeg', fps=5, dpi=200)

INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
INFO:matplotlib.animation:MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 3000x800 -pix_fmt rgba -framerate 5 -loglevel error -i pipe: -filter_complex 'split [a][b];[a] palettegen [p];[b][p] paletteuse' -y cell_animation.gif
